In [1]:
import sys
#import click
import numpy as np
import pandas as pd
import rpy2.robjects as ro
#from rpy2.rinterface import RRuntimeError
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from statsmodels.stats.multitest import multipletests

In [1]:
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources/R'

In [2]:
!which R

/usr/local/bin/R


In [10]:
#Import R libraries
base = importr('base')
stats = importr('stats')
limma = importr('limma')
writexl = importr('writexl')

PackageNotInstalledError: The R package "writexl" is not installed.